In [1]:
import tensorflow as tf
import numpy as np
import os


os.environ["CUDA_VISIBLE_DEVICES"]="1"


from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../MNIST_data", one_hot=True)



Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def Enc(x,isTrain = True, reuse = False, name = 'Enc') : #input = (minibatch * w * h * ch)
    
    # out size = (in size + 2*padding - kenel)/strides + 1    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)


    with tf.variable_scope('Enc',reuse=reuse) :
        
        #x = (-1, 1, 1, 100)

        conv1 = tf.layers.conv2d(x,64,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init) 
        r1 = tf.nn.elu(tf.layers.batch_normalization(conv1,training=isTrain))#14*14*64
        
        conv2 = tf.layers.conv2d(r1,128,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#7*7*128
                
        conv3 = tf.layers.conv2d(r2,256,[3,3], strides=(1,1),padding = 'valid',
                kernel_initializer=w_init, bias_initializer=b_init)
        r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#5*5*256

        conv4 = tf.layers.conv2d(r3,20,[5,5], strides=(1,1),padding = 'valid',
                kernel_initializer=w_init, bias_initializer=b_init)
    r4 = tf.layers.batch_normalization(conv4,training=isTrain, name = name)#1*1*100


    return r4
def Dec(x,isTrain=True,reuse = False, name='Dec') :
    
    # out size = (in size + 2*padding - kenel)/strides + 1    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)


    with tf.variable_scope('Dec',reuse=reuse) :
        
        #x = (-1, 1, 1, 100)

        conv1 = tf.layers.conv2d_transpose(x,256,[5,5], strides=(1,1),padding = 'valid',
                kernel_initializer=w_init, bias_initializer=b_init) 
        r1 = tf.nn.elu(tf.layers.batch_normalization(conv1,training=isTrain))#5*5*256
        
        conv2 = tf.layers.conv2d_transpose(r1,128,[3,3], strides=(1,1),padding = 'valid',
                kernel_initializer=w_init, bias_initializer=b_init)
        r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#7*7*128
                
        conv3 = tf.layers.conv2d_transpose(r2,64,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#14*14*64

        conv4 = tf.layers.conv2d_transpose(r3,1,[5,5], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
    r4 = tf.nn.sigmoid(conv4, name = name)#28*28*1

    return r4
def Disc(x,reuse = False, name = 'Disc') :
    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)
    with tf.variable_scope('Disc', reuse=reuse) :
        
        #x = (-1,64,64,1)
        # out size = (in size + 2*padding - kenel)/strides + 1   
        fc0  = tf.reshape(x, (-1, 20))
        
        w1 = tf.get_variable('w1',[20, 10],initializer=w_init)
        b1 = tf.get_variable('b1',[10],initializer=b_init)
        
                                          
    fc1 = tf.add(tf.matmul(fc0,w1) ,b1, name = name)
    
    return fc1




In [3]:

X = tf.placeholder(tf.float32,shape=(None,28,28,1),name = 'X')    

Y_Label = tf.placeholder(tf.float32,shape=(None,10),name = 'Y_Label')    

isTrain = tf.placeholder(dtype=tf.bool,name='isTrain')

enc = Enc(X,isTrain, reuse = False, name = 'enc') 
y = Dec(enc, isTrain, reuse = False, name= 'dec')

code = tf.sign(enc)
OutputLayer = Disc(code, name='disc')

classification_Loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = Y_Label, logits=OutputLayer))
regression_loss = tf.reduce_mean(tf.sqrt(tf.reduce_sum((X-y)**2, axis=[1,2,3])))       


correct_prediction = tf.equal(tf.argmax(OutputLayer,1), tf.argmax(Y_Label,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


T_vars = tf.trainable_variables()
Class_vars = [var for var in T_vars if var.name.startswith('Disc') ]
Regress_vars = [var for var in T_vars if var.name.startswith('Enc') or var.name.startswith('Dec') ]

    # When using the batchnormalization layers,
    # it is necessary to manually add the update operations
    # because the moving averages are not included in the graph
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)) :        
    Class_optim = tf.train.AdamOptimizer(2e-3).minimize(classification_Loss, var_list=Class_vars, name='Class_optim')     
    Regress_optim = tf.train.AdamOptimizer(2e-3).minimize(regression_loss, var_list=Regress_vars, name='Regress_optim')
    




In [4]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) 
sess.run(tf.global_variables_initializer())


for i  in range(10000) : 
    x_, y_ = mnist.train.next_batch(100)

    sess.run(Regress_optim,{X : np.reshape(x_,(-1,28,28,1)),isTrain : True})
    if i % 100 == 0 : 
        print(i, )




    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [5]:

for i  in range(10000) : 
    x_, y_ = mnist.train.next_batch(100)

    sess.run(Class_optim,{X : np.reshape(x_,(-1,28,28,1)),isTrain : False, Y_Label : y_})
    if i % 100 == 0 : 
        print(sess.run(accuracy,{X : np.reshape(mnist.test.images,(-1,28,28,1)), Y_Label : mnist.test.labels, isTrain : False}))

0.1028
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135
0.1135


KeyboardInterrupt: 